In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/lhg010524/Detection_Abandoned-vehicle'
dist = distutils.core.run_setup("./Detection_Abandoned-vehicle/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
import sys, os
sys.path.insert(0, os.path.abspath('./Detection_Abandoned-vehicle/detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 10.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15819, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 15819 (delta 31), reused 43 (delta 17), pack-reused 15743 (from 1)
Receiving objects: 100% (15819/15819), 6.39 MiB | 1.05 MiB/s, done.
Resolving deltas: 100% (11523/11523), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your e

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.5 ; cuda:  cu121
detectron2: 0.6


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
import os
import json
import cv2
import random
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# 어노테이션 파일 및 이미지 디렉토리 설정
annotation_files = [
    "/content/detectron2/1024_1.json",
    "/content/detectron2/1024_2.json",
    "/content/detectron2/1024_ver2_1.json",
    "/content/detectron2/1024_ver2_2.json",
    "/content/detectron2/1024_ver2_3.json",
    "/content/detectron2/1024_ver2_4.json",
    "/content/detectron2/256.json",
    "/content/detectron2/512.json"
]
image_dir = "/content/train"

# 모든 어노테이션 데이터를 병합
def load_annotations(annotation_files):
    merged_annotations = {"images": [], "annotations": [], "categories": []}
    annotation_id_offset = 0
    image_id_offset = 0
    image_id_map = {}

    for anno_file in annotation_files:
        with open(anno_file, "r", encoding="utf-8") as f:
            data = json.load(f)

            for img_data in data["images"]:
                new_image_id = image_id_offset + img_data["id"]
                image_id_map[img_data["id"]] = new_image_id
                img_data["id"] = new_image_id
                merged_annotations["images"].append(img_data)
            image_id_offset += len(data["images"])

            for anno_data in data["annotations"]:
                anno_data["id"] = annotation_id_offset + anno_data["id"]
                anno_data["image_id"] = image_id_map[anno_data["image_id"]]
                merged_annotations["annotations"].append(anno_data)
            annotation_id_offset += len(data["annotations"])

            if not merged_annotations["categories"]:
                merged_annotations["categories"] = data["categories"]

    return merged_annotations

# 이미지 파일 기준으로 어노테이션 데이터 필터링
def filter_annotations_by_image(image_dir, annotations):
    filtered_images = []
    filtered_annotations = []
    image_id_map = {}

    for img_data in annotations["images"]:
        img_path = os.path.join(image_dir, img_data["file_name"])
        if os.path.exists(img_path):
            filtered_images.append(img_data)
            image_id_map[img_data["id"]] = img_data

    for anno_data in annotations["annotations"]:
        if anno_data["image_id"] in image_id_map:
            filtered_annotations.append(anno_data)

    return {
        "images": filtered_images,
        "annotations": filtered_annotations,
        "categories": annotations["categories"],
    }

# 병합된 어노테이션 로드 및 필터링
merged_annotations = load_annotations(annotation_files)
filtered_data = filter_annotations_by_image(image_dir, merged_annotations)

# 데이터를 학습 및 검증으로 분리
def split_dataset(data, train_ratio=0.8):
    images = data["images"]
    random.shuffle(images)
    split_index = int(len(images) * train_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]

    train_image_ids = {img["id"] for img in train_images}
    train_annotations = [anno for anno in data["annotations"] if anno["image_id"] in train_image_ids]
    val_annotations = [anno for anno in data["annotations"] if anno["image_id"] not in train_image_ids]

    train_data = {
        "images": train_images,
        "annotations": train_annotations,
        "categories": data["categories"],
    }
    val_data = {
        "images": val_images,
        "annotations": val_annotations,
        "categories": data["categories"],
    }

    return train_data, val_data

train_data, val_data = split_dataset(filtered_data)

# 분리된 데이터 저장
train_annotation_path = "/content/train_annotations.json"
val_annotation_path = "/content/val_annotations.json"
with open(train_annotation_path, "w") as f:
    json.dump(train_data, f)
with open(val_annotation_path, "w") as f:
    json.dump(val_data, f)

# 데이터셋 등록
train_dataset_name = "vehicle_train"
val_dataset_name = "vehicle_val"
register_coco_instances(train_dataset_name, {}, train_annotation_path, image_dir)
register_coco_instances(val_dataset_name, {}, val_annotation_path, image_dir)

# 메타데이터 및 데이터셋 정보 가져오기
vehicle_train_metadata = MetadataCatalog.get(train_dataset_name)
vehicle_val_metadata = MetadataCatalog.get(val_dataset_name)

# 학습 데이터 시각화
dataset_dicts = DatasetCatalog.get(train_dataset_name)
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=vehicle_train_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

# 학습 구성 객체 생성
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# 데이터셋 설정
cfg.DATASETS.TRAIN = (train_dataset_name,)
cfg.DATASETS.TEST = (val_dataset_name,)

# 데이터 로더 설정
cfg.DATALOADER.NUM_WORKERS = 2

# 초기 가중치 설정
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# 학습 하이퍼파라미터 설정
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.000125
cfg.SOLVER.MAX_ITER = 6600         # 66
cfg.SOLVER.CHECKPOINT_PERIOD = 330  # 500 Iterations마다 체크포인트 저장

# RoIHeads 설정
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# 출력 디렉토리 설정
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# CustomTrainer 클래스 정의
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, output_dir=cfg.OUTPUT_DIR)

# 평가 주기 설정
cfg.TEST.EVAL_PERIOD = 330  # 원하는 평가 주기를 설정

# 학습 시작
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
import re
import numpy as np
import cv2
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

predictor = DefaultPredictor(cfg)

# 테스트할 이미지 불러오기
image_path = "/(37.479637°, 126.636221°).png"
im = cv2.imread(image_path)

# 이미지 제목에서 중심 위경도 추출
match = re.search(r"\(([\d.]+)°, ([\d.]+)°\)", image_path)
if match:
    lat_center = float(match.group(1))
    lon_center = float(match.group(2))
else:
    raise ValueError("이미지 제목에서 위경도를 추출할 수 없습니다.")

# 이미지 크기 및 실제 크기 정보
image_size = 1024  # 1024x1024 픽셀
real_size = 128  # 128x128 미터
pixel_size = real_size / image_size  # 0.125m/pixel

# 추론 수행
outputs = predictor(im)

# 바운딩 박스 중심 좌표 계산
boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
centers = [(int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)) for box in boxes]

# 위도 및 경도 변환 계산
lat_m_per_deg = 111320  # 위도 1도의 미터 단위
lon_m_per_deg = 111320 * np.cos(np.radians(lat_center))  # 경도 1도의 미터 단위

bounding_box_coords = []
print("Bounding Box Coordinates with numbering (latitude, longitude):")
for idx, center in enumerate(centers):
    pixel_dx = center[0] - image_size // 2  # 중심에서의 x 방향 픽셀 거리
    pixel_dy = center[1] - image_size // 2  # 중심에서의 y 방향 픽셀 거리

    # 실제 거리 변환
    real_dx = pixel_dx * pixel_size  # x 방향 실제 거리 (미터)
    real_dy = pixel_dy * pixel_size  # y 방향 실제 거리 (미터)

    # 위도/경도로 변환
    lat_offset = real_dy / lat_m_per_deg
    lon_offset = real_dx / lon_m_per_deg

    bbox_lat = lat_center + lat_offset
    bbox_lon = lon_center + lon_offset

    bounding_box_coords.append((bbox_lat, bbox_lon))
    print(f"Object {idx + 1}: Center at ({bbox_lat:.6f}, {bbox_lon:.6f})")

    # 이미지 위에 객체 넘버링 표시
    cv2.putText(
        im,
        f"{idx + 1}",
        (center[0], center[1]),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,  # 폰트 크기
        (0, 255, 0),  # 색상 (초록)
        2  # 두께
    )

# 출력 결과 시각화
v = Visualizer(im[:, :, ::-1], metadata=vehicle_metadata, scale=0.5)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Colab에서 이미지를 표시
cv2_imshow(out.get_image()[:, :, ::-1])
cv2_imshow(im)  # 넘버링된 원본 이미지 출력